In [1]:
# Pair Trading: KO vs PEP — 5 anos
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint
from scipy.stats import zscore

In [3]:
# 1. Baixar os dados (5 anos)
tickers = ['KO', 'PEP']
start = '2019-07-01'
end = '2024-07-01'
df = yf.download(tickers, start=start, end=end)['Close'].dropna()

C:\Users\Usuário\AppData\Local\Temp\ipykernel_10072\2731993309.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start=start, end=end)['Close'].dropna()
[*********************100%***********************]  2 of 2 completed


In [4]:
# 2. Teste de cointegração
score, pvalue, _ = coint(df['KO'], df['PEP'])
print(f'P-valor do teste de cointegração: {pvalue:.4f}')
if pvalue < 0.05:
    print("Os ativos são cointegrados — válido para Pair Trading")
else:
    print("Os ativos não são cointegrados — atenção!")

P-valor do teste de cointegração: 0.2878
Os ativos não são cointegrados — atenção!


In [5]:
# 3. Cálculo do spread e Z-score
spread = df['KO'] - df['PEP']
zscore_spread = zscore(spread)


In [6]:

# 4. Estratégia de entrada e saída
entry_z = 1.0
exit_z = 0.0
signals = pd.DataFrame(index=df.index)
signals['Z-Score'] = zscore_spread
signals['Position_KO'] = 0
signals['Position_PEP'] = 0

In [8]:
# Lógica de trade
signals.loc[signals['Z-Score'] > entry_z, ['Position_KO', 'Position_PEP']] = [-1, 1]  # Vende KO, compra PEP
signals.loc[signals['Z-Score'] < -entry_z, ['Position_KO', 'Position_PEP']] = [1, -1]  # Compra KO, vende PEP
signals.loc[abs(signals['Z-Score']) < exit_z, ['Position_KO', 'Position_PEP']] = [0, 0]  # Zera

# Preencher as posições ao longo do tempo
signals[['Position_KO', 'Position_PEP']] = signals[['Position_KO', 'Position_PEP']].ffill().fillna(0)


In [9]:
# 5. Cálculo dos retornos
returns = (signals[['Position_KO', 'Position_PEP']].shift(1) * df.pct_change()).sum(axis=1)
cumulative_returns = (1 + returns).cumprod()

# 6. Métricas
sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)
running_max = cumulative_returns.cummax()
drawdown = (cumulative_returns - running_max) / running_max
max_dd = drawdown.min()

C:\Users\Usuário\AppData\Local\Temp\ipykernel_10072\146003827.py:6: RuntimeWarning: invalid value encountered in scalar divide
  sharpe = np.mean(returns) / np.std(returns) * np.sqrt(252)


In [ ]:
# 7. Gráficos
plt.figure(figsize=(14, 8))

plt.subplot(3, 1, 1)
plt.plot(df['KO'], label='KO (Coca-Cola)')
plt.plot(df['PEP'], label='PEP (Pepsi)')
plt.legend()
plt.title('Preços históricos (5 anos)')

plt.subplot(3, 1, 2)
plt.plot(zscore_spread, label='Z-Score do Spread', color='purple')
plt.axhline(entry_z, color='red', linestyle='--')
plt.axhline(-entry_z, color='green', linestyle='--')
plt.axhline(0, color='black', linestyle='-')
plt.legend()
plt.title('Z-Score do Spread KO - PEP')

plt.subplot(3, 1, 3)
plt.plot(cumulative_returns, label='Curva de Capital', color='darkblue')
plt.title(f'Curva de Capital | Sharpe: {sharpe:.2f} | Max Drawdown: {max_dd:.2%}')
plt.legend()

plt.tight_layout()
plt.show()